In [94]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
import sys

sys.path.append("../")

In [96]:
import pandas as pd
import numpy as np

# Still getting "cannot import name 'NaN' from numpy" error
# https://github.com/twopirllc/pandas-ta/issues/857
import pandas_ta as ta

strategy_params = {
    "exchange_name": "binance",
    "mode": "live",
    "type": "future",
    "usd": 25,
    "account": "account_1",
    "symbol": "ETH/USDT",
    "count_start_date": "2024-01-01 00:00:00",
    "latest_start_date": "2025-01-01 00:00:00",
    "period": "1h",
    "count": 1000 * 10,
    "sma_a": 150,
    "sma_b": 130,
    "sma_c": 110,
    "atr_sl": 2.5,
    "atr_tp": 0,
    "atr_tsl": 2.5,
    "limit": True,
    "ratio": 0.3,
    "n_trials": 200,
    "enable_bot": True,
}

optuna_params = {
    "sma_a": {"min": 10, "max": 400, "step": 20, "type": "int"},
    "sma_b": {"min": 10, "max": 400, "step": 20, "type": "int"},
    "sma_c": {"min": 10, "max": 400, "step": 20, "type": "int"},
    "atr_sl": {"min": 1.0, "max": 4.0, "step": 0.5, "type": "float"},
    "atr_tsl": {"min": 1.0, "max": 4.0, "step": 0.5, "type": "float"},
}


def strategy(df, strategy_params):
    df["sma_a"] = ta.ema(df["close"], length=strategy_params["sma_a"])
    df["sma_b"] = ta.ema(df["close"], length=strategy_params["sma_b"])
    df["sma_c"] = ta.ema(df["close"], length=strategy_params["sma_c"])
    df["atr"] = ta.atr(df["high"], df["low"], df["close"], length=14)
    df["is_nan"] = np.isnan(df["sma_a"]) | np.isnan(df["sma_b"]) | np.isnan(df["atr"])

    df["long_enter"] = (
        (df["sma_a"] > df["sma_b"])
        & (df["sma_b"] > df["sma_c"])
        & (df["close"].shift() < df["sma_b"].shift())
        & (df["close"] > df["sma_b"])
        # & (df["sma_a"].shift() < df["sma_b"].shift())
    )
    df["long_status"] = -1
    df.loc[df["long_enter"], "long_status"] = 1
    df["long_idx"] = -1
    df["long_price"] = np.nan
    df["long_diff"] = np.nan
    df["long_total"] = 0.0
    df["long_sl"] = np.nan
    df["long_tp"] = np.nan
    df["long_tsl"] = np.nan
    # df.drop(["long_enter"], axis=1, inplace=True)

    df["short_enter"] = (
        (df["sma_a"] < df["sma_b"])
        & (df["sma_b"] < df["sma_c"])
        & (df["close"].shift() > df["sma_b"].shift())
        & (df["close"] < df["sma_b"])
        # & (df["sma_a"].shift() > df["sma_b"].shift())
    )
    df["short_status"] = -1
    df.loc[df["short_enter"], "short_status"] = 1
    df["short_idx"] = -1
    df["short_price"] = np.nan
    df["short_diff"] = np.nan
    df["short_total"] = 0.0
    df["short_sl"] = np.nan
    df["short_tp"] = np.nan
    df["short_tsl"] = np.nan
    # df.drop(["short_enter"], axis=1, inplace=True)

    df["merge_price"] = np.nan
    df["merge_diff"] = np.nan
    df["merge_total"] = 0.0

    # 关闭short
    # df["short_status"] = -1

    # df["long_status"] = -1


In [97]:
class Test_Strategy:
    def __init__(self, strategy_params, optuna_params, strategy):
        self.strategy_params = strategy_params
        self.optuna_params = optuna_params
        self.strategy = strategy


class Strategy2:
    def __init__(self, strategy_arr):
        self.strategy_arr = strategy_arr


ts = Test_Strategy(strategy_params, optuna_params, strategy)
strategy = Strategy2([["test_strategy", ts]])


In [98]:
import data_api
import backtest
import plot

# import strategy
import trade_api

import importlib

importlib.reload(data_api)
importlib.reload(backtest)
importlib.reload(plot)
# importlib.reload(strategy)
importlib.reload(trade_api)

from trade_api.trade_api import (
    trade_api_wapper,
    get_ticker,
    close_position_all,
    cancel_order_all,
    get_balance,
)
from data_api.data_api import get_data_wapper, load_config, test_connect_api


for [_name, _s] in strategy.strategy_arr:
    _s.strategy_params["mode"] = "live"

    period = _s.strategy_params["period"]
    print(f"run {period} {_name} {_s}")
    print(f"strategy_params {_s.strategy_params}")
    symbol = _s.strategy_params["symbol"]

    config_path = "../strategy/config.json"

    config = load_config(config_path)
    exchange = test_connect_api(config, _s.strategy_params)

    def _():
        symbol = strategy_params["symbol"]
        market = exchange.market(symbol)

        price = exchange.fetchTicker(symbol)["close"]

        limits = market["limits"]
        print(symbol, limits["cost"]["min"], limits["amount"]["min"])

    _()

    price = get_ticker(exchange, symbol)

    trade_api_wapper(
        exchange,
        _s.strategy_params,
        "buy",
        leverage=1,
        price=price,
        takeProfitPrice=price + 500,
        stopLossParams=price - 500,
        mode="open",
    )
    break

run 1h test_strategy <__main__.Test_Strategy object at 0x00000220AB3708F0>
strategy_params {'exchange_name': 'binance', 'mode': 'live', 'type': 'future', 'usd': 25, 'account': 'account_1', 'symbol': 'ETH/USDT', 'count_start_date': '2024-01-01 00:00:00', 'latest_start_date': '2025-01-01 00:00:00', 'period': '1h', 'count': 10000, 'sma_a': 150, 'sma_b': 130, 'sma_c': 110, 'atr_sl': 2.5, 'atr_tp': 0, 'atr_tsl': 2.5, 'limit': True, 'ratio': 0.3, 'n_trials': 200, 'enable_bot': True}
{'FDUSD': 0.0, 'BFUSD': 0.0, 'BNB': 0.0, 'ETH': 0.0, 'BTC': 0.0, 'USDT': 60.50032107, 'USDC': 0.0}
USDT {'free': 60.50032107, 'used': 0.0, 'total': 60.50032107}
ETH/USDT 20.0 0.001
trade_api_wapper: AuthenticationError binance {"code":-2015,"msg":"Invalid API-key, IP, or permissions for action"}


('TRUMP/USDT', 5.0, 0.01)

In [11]:
trade_api_wapper(exchange, _s.strategy_params, "buy", leverage=1, mode="close")

In [10]:
trade_api_wapper(exchange, _s.strategy_params, "buy", leverage=1, mode="cancel")


In [70]:
balance_obj = get_balance(exchange)

print(balance_obj["total"])  # 总资产数量

print("USDT", balance_obj.get("USDT", {}))
print("BTC", balance_obj.get("BTC", {}))
print("ETH", balance_obj.get("ETH", {}))
print("DOGE", balance_obj.get("DOGE", {}))

{'FDUSD': 0.0, 'BFUSD': 0.0, 'BNB': 0.0, 'ETH': 0.0, 'BTC': 0.0, 'USDT': 60.50032107, 'USDC': 0.0}
USDT {'free': 60.50032107, 'used': 0.0, 'total': 60.50032107}
BTC {'free': 0.0, 'used': 0.0, 'total': 0.0}
ETH {'free': 0.0, 'used': 0.0, 'total': 0.0}
DOGE {}
